<a href="https://colab.research.google.com/github/yckamra/MovieLens25m-Recommender/blob/main/MovieLens_Recommender_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
import zipfile
import os
import pandas as pd

In [2]:
auth.authenticate_user()

!gcloud config set project movielens-recommender-451017

!gsutil cp gs://movielens-data/movielens_data.zip /content/

Updated property [core/project].
Copying gs://movielens-data/movielens_data.zip...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

/ [1 files][257.5 MiB/257.5 MiB]                                                
Operation completed over 1 objects/257.5 MiB.                                    


Dataset can be found at: https://www.kaggle.com/datasets/garymk/movielens-25m-dataset

In [3]:
!ls -lh /content/

total 258M
-rw-r--r-- 1 root root 258M May 19 16:51 movielens_data.zip
drwxr-xr-x 1 root root 4.0K May 14 13:38 sample_data


In [4]:
zip_path = "/content/movielens_data.zip"  # Change to your actual zip file name
extract_to = "/content/data/"  # Where to extract files

# Create directory if it doesn't exist
os.makedirs(extract_to, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)

print("Extraction complete! Files are in:", extract_to)
!ls -lh /content/data/ml-25m

Extraction complete! Files are in: /content/data/
total 1.1G
-rw-r--r-- 1 root root 416M May 19 16:52 genome-scores.csv
-rw-r--r-- 1 root root  18K May 19 16:52 genome-tags.csv
-rw-r--r-- 1 root root 1.4M May 19 16:52 links.csv
-rw-r--r-- 1 root root 2.9M May 19 16:52 movies.csv
-rw-r--r-- 1 root root 647M May 19 16:52 ratings.csv
-rw-r--r-- 1 root root  11K May 19 16:51 README.txt
-rw-r--r-- 1 root root  38M May 19 16:52 tags.csv


In [5]:
genome_scores_CSV = "/content/data/ml-25m/genome-scores.csv"
genome_tags_CSV = "/content/data/ml-25m/genome-tags.csv"
links_CSV = "/content/data/ml-25m/links.csv"
movies_CSV = "/content/data/ml-25m/movies.csv"
ratings_CSV = "/content/data/ml-25m/ratings.csv"
tags_CSV = "/content/data/ml-25m/tags.csv"

In [6]:
# Load ratings data
ratings_path = ratings_CSV
ratings_df = pd.read_csv(ratings_path)

# Count unique users and movies
num_users = ratings_df["userId"].nunique()
num_movies = ratings_df["movieId"].nunique()

print(f"Number of Unique Users: {num_users}")
print(f"Number of Unique Movies: {num_movies}")

Number of Unique Users: 162541
Number of Unique Movies: 59047
